<a href="https://colab.research.google.com/github/choi-yh/DataMining/blob/master/8_2_rnn_many_to_one_TF2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* many to one RNN은 단어 시퀀스를 판별하는 문제로 예를 들어, 영화평을 입력하면 부정인지 긍정인지 판단하는 것.  
* 문장은 Tokenization을 통해 단어들의 모음으로 변환되고 이는 다시 one-hot encoding을 통해 수치로 변환된다.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Preparing dataset

* words의 모든 문자를 join하고 set에 저장한 후, 다시 리스트에 저장한다.  
이후 <pad> 라는 "0"에 해당하는 문자를 삽입한다.  
이후, index -> character, character -> index로 반환하는 딕셔너리를 생성한다.

In [ ]:
# example data
words = ['good', 'bad', 'worse', 'so good']
y_data = [[1], [0], [0], [1]]

# creating a token dictionary
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx: char for idx, char in enumerate(char_set)}
char2idx = {char: idx for idx, char in enumerate(char_set)}

print(char_set)
print(idx2char)
print(char2idx)

['<pad>', ' ', 'a', 'b', 'd', 'e', 'g', 'o', 'r', 's', 'w']
{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [ ]:
# converting sequence of tokens to sequence of indices
x_data = list(map(lambda word: [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word: len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [ ]:
# x_data의 길이가 다른 것을 보완하기 위해 padding을 사용
# padding the sequence of indices
max_sequence = 10
x_data = pad_sequences(sequences=x_data, maxlen=max_sequence,
                       padding='post', truncating='post')
y_data = np.array(y_data)

# checking data
print(x_data)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[[1]
 [0]
 [0]
 [1]]


### Creating model

In [ ]:
# creating simple rnn for "many to one" classification
input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx))
hidden_size = 10
num_classes = 2

model = Sequential()
model.add(layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                           mask_zero=True, input_length=max_sequence,
                           embeddings_initializer=keras.initializers.Constant(one_hot)))
model.add(layers.SimpleRNN(units=hidden_size))
model.add(layers.Dense(units=num_classes))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 11)            121       
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 10)                220       
_________________________________________________________________
dense (Dense)                (None, 2)                 22        
Total params: 363
Trainable params: 363
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_data

array([[ 6,  7,  7,  4,  0,  0,  0,  0,  0,  0],
       [ 3,  2,  4,  0,  0,  0,  0,  0,  0,  0],
       [10,  7,  8,  9,  5,  0,  0,  0,  0,  0],
       [ 9,  7,  1,  6,  7,  7,  4,  0,  0,  0]], dtype=int32)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_data, y_data, epochs=30)

Epoch 1/30
1/1 [==============================] - 0s 1ms/step - loss: 3.9692 - accuracy: 0.7500
Epoch 2/30
1/1 [==============================] - 0s 1ms/step - loss: 3.9544 - accuracy: 0.7500
Epoch 3/30
1/1 [==============================] - 0s 1ms/step - loss: 3.9387 - accuracy: 0.7500
Epoch 4/30
1/1 [==============================] - 0s 2ms/step - loss: 3.9216 - accuracy: 0.7500
Epoch 5/30
1/1 [==============================] - 0s 1ms/step - loss: 3.9032 - accuracy: 0.7500
Epoch 6/30
1/1 [==============================] - 0s 1ms/step - loss: 3.8830 - accuracy: 0.7500
Epoch 7/30
1/1 [==============================] - 0s 1ms/step - loss: 3.8608 - accuracy: 0.7500
Epoch 8/30
1/1 [==============================] - 0s 1ms/step - loss: 3.8361 - accuracy: 0.7500
Epoch 9/30
1/1 [==============================] - 0s 1ms/step - loss: 3.8084 - accuracy: 0.7500
Epoch 10/30
1/1 [==============================] - 0s 1ms/step - loss: 3.7768 - accuracy: 0.7500
Epoch 11/30
1/1 [======================

In [ ]:
yhat = model.predict(x_data)
yhat = np.argmax(yhat, axis=-1)
print(y_data, yhat)

[[1]
 [0]
 [0]
 [1]] [1 0 0 1]
